In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model, Sequential, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.applications import MobileNetV2, VGG16, DenseNet121
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define image dimensions
img_height, img_width = 224, 224

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/train_test_val/train',
                                                    target_size=(img_height, img_width),
                                                    batch_size=16,
                                                    class_mode='binary')
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/train_test_val/val',
                                                target_size=(img_height, img_width),
                                                batch_size=16,
                                                class_mode='binary')
test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/train_test_val/test',
                                                 target_size=(img_height, img_width),
                                                 batch_size=16,
                                                 class_mode='binary',
                                                 shuffle=False)

# Define checkpoint
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', save_best_only=True, mode='max')

########################################
# MobileNetV2 Model
########################################
mobilenet_v2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in mobilenet_v2.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(mobilenet_v2.output)
x = Dropout(0.5)(x)
prediction = Dense(1, activation='sigmoid')(x)
model1 = Model(inputs=mobilenet_v2.input, outputs=prediction)
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history1 = model1.fit(train_generator, validation_data=val_generator, epochs=20, callbacks=[checkpoint])
model1.save('/content/drive/MyDrive/model/mobileNetV2_ensemble_model.h5')

# Plot MobileNetV2
plt.plot(history1.history['accuracy'], label='Train')
plt.plot(history1.history['val_accuracy'], label='Validation')
plt.title('MobileNetV2 Accuracy')
plt.legend()
plt.show()

plt.plot(history1.history['loss'], label='Train')
plt.plot(history1.history['val_loss'], label='Validation')
plt.title('MobileNetV2 Loss')
plt.legend()
plt.show()

########################################
# VGG16 Model
########################################
vgg_16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
for layer in vgg_16.layers:
    layer.trainable = False

x = Flatten()(vgg_16.output)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
prediction = Dense(1, activation='sigmoid')(x)
model2 = Model(inputs=vgg_16.input, outputs=prediction)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history2 = model2.fit(train_generator, validation_data=val_generator, epochs=30, callbacks=[checkpoint])

# Plot VGG16
plt.plot(history2.history['accuracy'], label='Train')
plt.plot(history2.history['val_accuracy'], label='Validation')
plt.title('VGG16 Accuracy')
plt.legend()
plt.show()

plt.plot(history2.history['loss'], label='Train')
plt.plot(history2.history['val_loss'], label='Validation')
plt.title('VGG16 Loss')
plt.legend()
plt.show()

########################################
# DenseNet121 Model
########################################
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
densenet.trainable = False

model3 = Sequential()
model3.add(densenet)
model3.add(GlobalAveragePooling2D())
model3.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model3.add(Dropout(0.5))
model3.add(Dense(1, activation='sigmoid'))

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model3.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history3 = model3.fit(train_generator, validation_data=val_generator, epochs=20, verbose=2, callbacks=[early_stopping])
model3.save('/content/drive/MyDrive/Colab Notebooks/train_test_val/Dense121_64_model.h5')

# Plot DenseNet121
plt.plot(history3.history['accuracy'], label='Train')
plt.plot(history3.history['val_accuracy'], label='Validation')
plt.title('DenseNet121 Accuracy')
plt.legend()
plt.show()

plt.plot(history3.history['loss'], label='Train')
plt.plot(history3.history['val_loss'], label='Validation')
plt.title('DenseNet121 Loss')
plt.legend()
plt.show()